In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from typing import TypedDict

class State(TypedDict):
    graph_state: str

In [ ]:
def node1(state):
    print('---N1---')
    return {'graph_state': state['graph_state']+'I am'}

def node2(state):
    print('---N2---')
    return {'graph_state': state['graph_state']+'happy'}

def node3(state):
    print('---N3---')
    return {'graph_state': state['graph_state']+'sad'}


In [ ]:
import random
from typing import Literal

def mode(state)->Literal['node2','node3']:
    current_state = state['graph_state']

    if random.random() <.5:
        return 'node2'
    return 'node3'

In [48]:
from langgraph.graph import StateGraph,START,END

builder= StateGraph(State)

builder.add_node('node1',node1)
builder.add_node('node2',node2)
builder.add_node('node3',node3)

builder.add_edge(START,'node1')
builder.add_conditional_edges('node1',mode)
builder.add_edge('node2',END)
builder.add_edge('node3',END)

graph= builder.compile()


In [50]:
graph.invoke({'graph_state':'Hi I am Rt'})

---N1---
---N2---


{'graph_state': 'Hi I am RtI amhappy'}